# 연구 요약
---
## 이전 주 연구에서 부족했던 부분의 점검.
> 2020/05/07 “mnist 실시간 숫자 손글씨 검출”

* 영상처리를 이용한 전처리 과정을 일부 수정

* 예측에 사용되는 모델을 교체 (이전 주에 사용했던 MLP외에 다른 것을 사용)


# 이전 주 모델에서 부족했던 점
---
* 사용한 모델이 너무 단순하였음
    * 지나치게 단순한 MLP를 사용

* 숫자 영상의 정규화가 불충분하게 이루어짐
    * 손글씨 영상 데이터 수집 설계부터 과정까지 부실하였음


# 개선사항
---
* 윤곽선 검출을 이용하여, 프레임으로부터 “숫자를 검출하고 → 잘라내어 → 정규화”
    * ⇒ 하나의 프레임에서 여러 개 숫자를 동시에 검출할 수 있음.

* 2D Convolution을 사용하는 모델 사용
    * ⇒ 모델의 복잡도를 높임


# 소스코드 구성
---
* 손글씨 검출
    * 모든 손글씨 검출 : 붉은 색상에 대한 마스크 이미지 생성
    * 각 손글씨 외곽선 검출 : 각 손글씨에 대한 마스크 이미지 생성
    * 각 손글씨 추출
* 모델을 사용하여 숫자 예측




## 손 글씨 검출 과정

### 1. 모든 손 글씨 검출

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

# 모델을 바꾸려면 여기서 수정. (레포지토리의 'models/' 디렉토리에서 선택)

from models.mnist import model, checkpoint, input_shape

def createMask_red(frame):
    # 빨간 색상을 찾고, 빨간색 영역에 대한 마스크이미지를 생성한다.
    yuv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
    red_yuv_mask = cv2.inRange(yuv_frame,
                                ( 64,  0,128),
                                (255,128,255))
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    red_hsv_mask = cv2.inRange(hsv_frame,
                                (  0, 48, 48),
                                (255,255,255))
    return cv2.bitwise_and(red_yuv_mask, red_hsv_mask)


model.load_weights(checkpoint)
vidIn = cv2.VideoCapture(0)

# 적절한 커널 크기 결정
ret, frame = vidIn.read()
kernel_size = tuple([2 * (min(frame.shape[:1]) // 240) + 1] * 2)

# ===========================================================
# Main loop
# ===========================================================
while vidIn.isOpened():
    ret, frame = vidIn.read()
    if not ret: break

    # -----------------------------------------------------------
    # Detect hand writings
    # -----------------------------------------------------------
    red_mask = createMask_red(frame)
    red_mask = cv2.morphologyEx(red_mask,
                                cv2.MORPH_CLOSE,
                                cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size),
                                iterations=3)
    contours, hierarchy = cv2.findContours(red_mask,
                                            cv2.RETR_EXTERNAL,
                                            cv2.CHAIN_APPROX_SIMPLE)
    for num_cont in contours:
        # TODO: 각 컨투어(=숫자)에 대한 28x28 정규화 된 이미지 생성 루틴
        pass

### 2. 개별 손 글씨 추출 및 정규화

In [ ]:
x_data = []

for num_cont in contours:
    x,y,w,h = cv2.boundingRect(num_cont)

    frame = cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255))

    # 프레임으로 부터 숫자를 잘라냄 (num_masked)
    num_mask = np.zeros(red_mask.shape, dtype=np.uint8)
    cv2.drawContours(num_mask, [num_cont], 0, 255, -1)
    num_masked = cv2.bitwise_and(red_mask,red_mask, mask=num_mask)[y:y+h, x:x+w]

    # 잘라낸 숫자를 (dx,dy)만큼 평행이동하여, 원점 근처로 옮김.
    #   이후 과정에서 새로운 '정사각형' 프레임의 중앙에 위치시키기 위해, 적절히 dx, dy를 조정
    dx, dy = 0, 0
    if (w > h): dy = (w-h)//2
    else:       dx = (h-w)//2

    # 새로 생성한 정사각형 형태의 프레임에 삽입 : num_frame
    n = max([h,w])
    num_frame = np.zeros((n,n), dtype=np.uint8)
    num_frame[dy:dy+h, dx:dx+w] = num_masked

    # input_shape의 형태에 맞게 맞추어주기. (28x28[x1 ...])
    _ksize = tuple([2*(n//72)+1] * 2) # 홀수를 유지, 숫자 프레임의 너비 비율에 맞게 적절한 커널 사이즈 지정
    num_frame = cv2.dilate(num_frame, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, _ksize))
    num_frame = cv2.resize(num_frame, (28, 28))
    num_frame = num_frame.reshape(input_shape) # 모델이 확장된 차원을 사용하는 경우에 대한 처리 (ex. Conv2D)

    x_data.append(num_frame)


---

# 결론

## 해석
## 고찰
## 계획

---

# 부록

---

# 세미나

## 연구원이름

### 분류: {Paper Scanning / Study / Introduce / Feedback}
### 주제: 

# 전체 코드
---

In [ ]:
# -----------------------------------------------------------
# demonstrate how to classify multiple hand writings from videos
# being captured in real-time, using opencv-python
# 
# (C) 2020 Kim Dong Joo, Dongguk University, Gyeongju
# email hepheir@gmail.com
# -----------------------------------------------------------

import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

# 모델을 바꾸려면 여기서 수정. (레포지토리의 'models/' 디렉토리에서 선택)

from models.mnist import model, checkpoint, input_shape
# from models.mnist_2_layers import model, checkpoint, input_shape

def writeText(frame, txt, color=(128, 128, 255), pos=(32,16)):
    cv2.putText(frame,
                txt,
                pos,               # Coordinates
                cv2.FONT_HERSHEY_PLAIN, # 
                1.2,                    # Font scale
                color,        # Font color
                lineType=cv2.LINE_AA)

def getNumericKey(key):
    # 0~9사이의 키가 눌렸을 때만 해당 키를 반환.
    # 그 외의 키가 눌리면 None을 반환한다.
    c = chr(key)
    if c in "0123456789":
        return int(c)
    return None

def createMask_red(frame):
    # 빨간 색상을 찾고, 빨간색 영역에 대한 마스크이미지를 생성한다.
    yuv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
    red_yuv_mask = cv2.inRange(yuv_frame,
                                ( 64,  0,128),
                                (255,128,255))
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    red_hsv_mask = cv2.inRange(hsv_frame,
                                (  0, 48, 48),
                                (255,255,255))
    return cv2.bitwise_and(red_yuv_mask, red_hsv_mask)

def save_data(x_test, y_test):
    # 학습(또는 테스트)데이터와 예측값을 지정된 디렉토리안에 저장한다.
    path = 'out/'
    status = [0] * 10
    for i in range(len(x_test)):
        x, y = np.squeeze(x_test[i]), y_test[i]

        fname = '%s_%s_%d.png' % (str(y), str(cv2.getTickCount()), status[y])
        status[y] += 1

        cv2.imwrite(path + fname, x)

    print('files saved at ' + path)

if __name__ == '__main__':
    model.load_weights(checkpoint)

    vidIn = cv2.VideoCapture(0)
    # vidIn = cv2.VideoCapture('res/raw/TEST.MOV')

    # 적절한 커널 크기 결정
    ret, frame = vidIn.read()
    kernel_size = tuple([2 * (min(frame.shape[:1]) // 240) + 1] * 2) # 그냥 적절히 정한 마법의 숫자.

    # ===========================================================
    # Main loop
    # ===========================================================
    while vidIn.isOpened():
        ret, frame = vidIn.read()
        if not ret: break

        # -----------------------------------------------------------
        # Detect hand writings
        # -----------------------------------------------------------
        red_mask = createMask_red(frame)
        red_mask = cv2.morphologyEx(red_mask,
                                    cv2.MORPH_CLOSE,
                                    cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size),
                                    iterations=3)
        contours, hierarchy = cv2.findContours(red_mask,
                                               cv2.RETR_EXTERNAL,
                                               cv2.CHAIN_APPROX_SIMPLE)

        # -----------------------------------------------------------
        # Normalize each hand writing
        # -----------------------------------------------------------
        x_data = []
        box    = []

        for num_cont in contours:
        # 각 컨투어(=숫자)에 대한 28x28 정규화 된 이미지 생성 루틴
            x,y,w,h = cv2.boundingRect(num_cont)

            frame = cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255))

            # 프레임으로 부터 숫자를 잘라냄 (num_masked)
            num_mask = np.zeros(red_mask.shape, dtype=np.uint8)
            cv2.drawContours(num_mask, [num_cont], 0, 255, -1)
            num_masked = cv2.bitwise_and(red_mask,red_mask, mask=num_mask)[y:y+h, x:x+w]

            # 잘라낸 숫자를 (dx,dy)만큼 평행이동하여, 원점 근처로 옮김.
            #   이후 과정에서 새로운 '정사각형' 프레임의 중앙에 위치시키기 위해, 적절히 dx, dy를 조정
            dx, dy = 0, 0
            if (w > h): dy = (w-h)//2
            else:       dx = (h-w)//2
            
            # 새로 생성한 정사각형 형태의 프레임에 삽입 : num_frame
            n = max([h,w])
            num_frame = np.zeros((n,n), dtype=np.uint8)
            num_frame[dy:dy+h, dx:dx+w] = num_masked

            # input_shape의 형태에 맞게 맞추어주기. (28x28[x1 ...])
            _ksize = tuple([2*(n//72)+1] * 2) # 홀수를 유지, 숫자 프레임의 너비 비율에 맞게 적절한 커널 사이즈 지정
            num_frame = cv2.dilate(num_frame, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, _ksize))
            num_frame = cv2.resize(num_frame, (28, 28))
            num_frame = num_frame.reshape(input_shape) # 모델이 확장된 차원을 사용하는 경우에 대한 처리 (ex. Conv2D)

            x_data.append(num_frame)
            box.append([x,y,w,h])

        x_data = np.array(x_data)

        # -----------------------------------------------------------
        # Predict using pre-trained model
        # -----------------------------------------------------------
        if len(x_data):
            # 모든 숫자에 대해 한 번에 예측
            p = model.predict(x_data)
            y_data = [np.where(_p == max(_p))[0][0] for _p in p]

            for i in range(len(x_data)):
                # 예측된 값의 확인을 용이하게 하기위해, 각 숫자 바운딩 박스 주변에 예측된 값을 출력
                x,y,w,h = box[i]
                if max(p[i]) < 1: # 불확실한 예측에는 노랑색 사용
                    writeText(frame, str(y_data[i]), color=(0,255,255), pos=(x, y-4))
                else:
                    writeText(frame, str(y_data[i]), pos=(x,y-4))

        # -----------------------------------------------------------
        # Key mappings
        # -----------------------------------------------------------
        key = cv2.waitKey(20) & 0xFF
        if key == 27: break # ESC
        if key == ord('s'): save_data(x_data, y_data)

        # -----------------------------------------------------------
        # Re-train model (if the prediction is wrong)
        # -----------------------------------------------------------
        # # TODO: 재학습 과정은 나중에 생각.
        # x_train = x
        # y_train = np.expand_dims(getNumericKey(key), axis=0)

        # if y_train[0] != None: # 레이블이 주어지면 학습
        #     model.fit(x_train, y_train, epochs=1)

        # -----------------------------------------------------------
        # Show the result on screen
        # -----------------------------------------------------------
        cv2.imshow('Camera', frame)

    vidIn.release()
    cv2.destroyAllWindows()

print("")
print("Successfully Closed")